# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [2]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
# TODO: Import the necessary libs - DONE
import os
from typing import List, Dict, Any, Optional, TypedDict
from dotenv import load_dotenv
import chromadb
from chromadb.utils import embedding_functions
from tavily import TavilyClient

from lib.agents import Agent, AgentState
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import Tool
from lib.state_machine import StateMachine, Step, EntryPoint, Termination, Run

In [4]:
# TODO: Load environment variables - DONE
load_dotenv()

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# Initialize Clients
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(api_key=os.getenv("OPENAI_API_KEY"))
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_collection(name="udaplay", embedding_function=embedding_fn)
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [5]:
# TODO: Create retrieve_game tool - DONE
def retrieve_game(query: str) -> str:
    """
    Semantic search: Finds most results in the vector DB
    """
    print(f"DEBUG: Retrieving for query: {query}")
    results = collection.query(query_texts=[query], n_results=3)
    
    docs = results['documents'][0]
    metas = results['metadatas'][0]
    
    context = ""
    for doc, meta in zip(docs, metas):
        context += f"Game: {meta.get('Name')} ({meta.get('Platform')}, {meta.get('YearOfRelease')})\nDetails: {doc}\n\n"
    return context

retrieve_tool = Tool(retrieve_game)

#### Evaluate Retrieval Tool

In [6]:
# TODO: Create evaluate_retrieval tool - DONE
def evaluate_retrieval(question: str, retrieved_docs: str) -> str:
    """
    Based on the user's question and on the list of retrieved documents, 
    it will analyze the usability of the documents to respond to that question. 
    """
    print(f"DEBUG: Evaluating context")
    llm = LLM(model="gpt-3.5-turbo")
    prompt = f"""
    Question: {question}
    Retrieved Docs: {retrieved_docs}
    
    Are these docs useful to answer the question? Respond with ONLY 'YES' or 'NO'.
    """
    response = llm.invoke([UserMessage(content=prompt)])
    return response.content.strip().upper()

evaluate_tool = Tool(evaluate_retrieval)

#### Game Web Search Tool

In [7]:
# TODO: Create game_web_search tool - DONE
def game_web_search(question: str) -> str:
    """
    Semantic search: Finds most results in the vector DB (Fallback to web)
    """
    print(f"DEBUG: Web searching for: {question}")
    try:
        response = tavily_client.search(question, search_depth="advanced")
        context = ""
        for result in response['results']:
            context += f"Title: {result['title']}\nContent: {result['content']}\nURL: {result['url']}\n\n"
        return context
    except Exception as e:
        return f"Error searching web: {e}"

search_tool = Tool(game_web_search)

### Agent

In [10]:
# TODO: Create your Agent abstraction using StateMachine - DONE

class ResearchAgentState(AgentState):
    retrieved_context: Optional[str]
    evaluation_result: Optional[str]
    source: Optional[str]

class ResearchAgent(Agent):
    def __init__(self, model_name="gpt-3.5-turbo", temperature=0.7):
        super().__init__(model_name, "You are a game research assistant.", temperature=temperature)
        
    def _retrieve_step(self, state: ResearchAgentState) -> ResearchAgentState:
        query = state["user_query"]
        context = retrieve_game(query)
        return {**state, "retrieved_context": context, "source": "internal"}

    def _evaluate_step(self, state: ResearchAgentState) -> ResearchAgentState:
        query = state["user_query"]
        context = state["retrieved_context"]
        result = evaluate_retrieval(query, context)
        return {**state, "evaluation_result": result}

    def _web_search_step(self, state: ResearchAgentState) -> ResearchAgentState:
        query = state["user_query"]
        context = game_web_search(query)
        return {**state, "retrieved_context": context, "source": "web"}

    def _generate_answer_step(self, state: ResearchAgentState) -> ResearchAgentState:
        query = state["user_query"]
        context = state["retrieved_context"]
        source = state["source"]
        
        prompt_content = f"""
        Answer the question based ONLY on the context provided below.
        
        Context ({source}): 
        {context}
        
        Question: {query}
        
        Cite the source ({source}) in your answer.
        """
        
        # Combine history with the new prompt
        new_user_message = UserMessage(content=prompt_content)
        messages_for_llm = state["messages"] + [new_user_message]
        
        llm = LLM(model=self.model_name, temperature=self.temperature)
        response = llm.invoke(messages_for_llm)
        
        return {**state, "messages": state["messages"] + [new_user_message, AIMessage(content=response.content)]}

    def _create_state_machine(self) -> StateMachine[ResearchAgentState]:
        machine = StateMachine[ResearchAgentState](ResearchAgentState)
        
        entry = EntryPoint[ResearchAgentState]()
        retrieve = Step[ResearchAgentState]("retrieve", self._retrieve_step)
        evaluate = Step[ResearchAgentState]("evaluate", self._evaluate_step)
        web_search = Step[ResearchAgentState]("web_search", self._web_search_step)
        generate = Step[ResearchAgentState]("generate", self._generate_answer_step)
        termination = Termination[ResearchAgentState]()
        
        machine.add_steps([entry, retrieve, evaluate, web_search, generate, termination])
        
        machine.connect(entry, retrieve)
        machine.connect(retrieve, evaluate)
        
        def check_eval(state: ResearchAgentState) -> str:
            return "generate" if "YES" in state["evaluation_result"] else "web_search"
        
        machine.connect(evaluate, [generate, web_search], condition=check_eval)
        machine.connect(web_search, generate)
        machine.connect(generate, termination)
        
        return machine

In [14]:
# TODO: Invoke your agent - DONE
agent = ResearchAgent()

queries = [
    "When Pokémon Gold and Silver was released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X realeased for Playstation 5?"
]

for query in queries:
    print(f"\n--- Query: {query} ---")
    run = agent.invoke(query)
    print(f"Answer: {run.get_final_state()['messages'][-1].content}")


--- Query: When Pokémon Gold and Silver was released? ---
[StateMachine] Starting: __entry__
DEBUG: Retrieving for query: When Pokémon Gold and Silver was released?
[StateMachine] Executing step: retrieve
DEBUG: Evaluating context
[StateMachine] Executing step: evaluate
DEBUG: Web searching for: When Pokémon Gold and Silver was released?
[StateMachine] Executing step: web_search
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
Answer: Pokémon Gold and Silver were released in Japan on November 21, 1999.
Source: https://www.pokemon.com/us/pokemon-video-games/pokemon-gold-version-and-pokemon-silver-version

--- Query: Which one was the first 3D platformer Mario game? ---
[StateMachine] Starting: __entry__
DEBUG: Retrieving for query: Which one was the first 3D platformer Mario game?
[StateMachine] Executing step: retrieve
DEBUG: Evaluating context
[StateMachine] Executing step: evaluate
DEBUG: Web searching for: Which one was the first 3D platformer Mar

### (Optional) Advanced

In [15]:
# TODO: Update your agent with long-term memory - DONE
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes - DONE

class LearningResearchAgent(ResearchAgent):
    def _generate_answer_step(self, state: ResearchAgentState) -> ResearchAgentState:
        # Generate answer using parent logic
        new_state = super()._generate_answer_step(state)
        
        # Long-term memory: Save web results to ChromaDB
        if new_state["source"] == "web":
            print(f"DEBUG: Memorizing new info about '{new_state['user_query']}'")
            try:
                collection.add(
                    ids=[f"memory_{abs(hash(new_state['user_query']))}"],
                    documents=[f"Q: {new_state['user_query']}\nA: {new_state['messages'][-1].content}"],
                    metadatas=[{"Name": "Learned Memory", "Platform": "Web", "YearOfRelease": 2025}]
                )
            except Exception as e:
                print(f"DEBUG: Memory update skipped: {e}")
        return new_state

# Test the learning agent
learning_agent = LearningResearchAgent()
learning_agent.invoke("What is the release date of GTA 6?")

[StateMachine] Starting: __entry__
DEBUG: Retrieving for query: What is the release date of GTA 6?
[StateMachine] Executing step: retrieve
DEBUG: Evaluating context
[StateMachine] Executing step: evaluate
DEBUG: Web searching for: What is the release date of GTA 6?
[StateMachine] Executing step: web_search
DEBUG: Memorizing new info about 'What is the release date of GTA 6?'
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__


Run('335fbdc5-d717-4698-9cef-abde152d5cff')

In [11]:
# Initialize Agent
agent = ResearchAgent()

# Test Queries
queries = [
    "Who developed Gran Turismo?",
    "When was God of War Ragnarok released?",
    "What is the latest news about GTA 6?"
]

for query in queries:
    print(f"\n--- Query: {query} ---")
    run = agent.invoke(query)
    final_state = run.get_final_state()
    print(f"Final Answer:\n{final_state['messages'][-1].content}")
    print(f"Source: {final_state.get('source')}")


--- Query: Who developed Gran Turismo? ---
[StateMachine] Starting: __entry__
DEBUG: Retrieving for query: Who developed Gran Turismo?
[StateMachine] Executing step: retrieve
DEBUG: Evaluating context
[StateMachine] Executing step: evaluate
DEBUG: Web searching for: Who developed Gran Turismo?
[StateMachine] Executing step: web_search
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
Final Answer:
Polyphony Digital developed Gran Turismo. 
Source: https://gran-turismo.fandom.com/wiki/Polyphony_Digital
Source: web

--- Query: When was God of War Ragnarok released? ---
[StateMachine] Starting: __entry__
DEBUG: Retrieving for query: When was God of War Ragnarok released?
[StateMachine] Executing step: retrieve
DEBUG: Evaluating context
[StateMachine] Executing step: evaluate
DEBUG: Web searching for: When was God of War Ragnarok released?
[StateMachine] Executing step: web_search
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termi

In [12]:
# Demonstrate Session Management
print("\n=== Session Management Demo ===")

# Session 1: Ask about a specific game
print("\n--- Session 1: Context Building ---")
agent.invoke("Tell me about The Last of Us.", session_id="session_1")
run1 = agent.invoke("Who is the main character?", session_id="session_1")
print(f"Session 1 Answer: {run1.get_final_state()['messages'][-1].content}")

# Session 2: Ask about a different game
print("\n--- Session 2: Independent Context ---")
agent.invoke("Tell me about Super Mario.", session_id="session_2")
run2 = agent.invoke("Who is the main character?", session_id="session_2")
print(f"Session 2 Answer: {run2.get_final_state()['messages'][-1].content}")

# Verify Session 1 context is preserved
print("\n--- Session 1: Context Verification ---")
run3 = agent.invoke("What year was it released?", session_id="session_1")
print(f"Session 1 Follow-up Answer: {run3.get_final_state()['messages'][-1].content}")


=== Session Management Demo ===

--- Session 1: Context Building ---
[StateMachine] Starting: __entry__
DEBUG: Retrieving for query: Tell me about The Last of Us.
[StateMachine] Executing step: retrieve
DEBUG: Evaluating context
[StateMachine] Executing step: evaluate
DEBUG: Web searching for: Tell me about The Last of Us.
[StateMachine] Executing step: web_search
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[StateMachine] Starting: __entry__
DEBUG: Retrieving for query: Who is the main character?
[StateMachine] Executing step: retrieve
DEBUG: Evaluating context
[StateMachine] Executing step: evaluate
DEBUG: Web searching for: Who is the main character?
[StateMachine] Executing step: web_search
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
Session 1 Answer: The main character in a story is known as the protagonist. They are the character who drives the action and whose fate matters most in the narrative.

Sou